Get official evaluation script 

In [15]:
!git clone https://bitbucket.org/HBrum/tweetsentbr.git
%cd tweetsentbr/sent-analysis/
!pip install nlpnet

Cloning into 'tweetsentbr'...
remote: Counting objects: 85, done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 85 (delta 20), reused 0 (delta 0)
Unpacking objects: 100% (85/85), done.
/content/mt-dnn_port/tweetsentbr/sent-analysis


Extract dataset

In [16]:
!unzip tweetSentBR_extracted.zip

Archive:  tweetSentBR_extracted.zip
   creating: tweetSentBR_extracted/
  inflating: tweetSentBR_extracted/testTT.neg  
  inflating: tweetSentBR_extracted/testTT.neu  
  inflating: tweetSentBR_extracted/testTT.pos  
  inflating: tweetSentBR_extracted/trainTT.neg  
  inflating: tweetSentBR_extracted/trainTT.neu  
  inflating: tweetSentBR_extracted/trainTT.pos  
  inflating: tweetSentBR_extracted/tweets.none  
  inflating: tweetSentBR_extracted/tweets.neg  
  inflating: tweetSentBR_extracted/tweets.neu  
  inflating: tweetSentBR_extracted/tweets.pos  


Get structered dataset

In [17]:
from io import StringIO
import pandas as pd
import os
import re

corpora = [f for f in os.listdir('tweetSentBR_extracted') if 'testTT' in f]

print('Separating lines in tabs...')
tabbed = dict()
for goldpath in corpora:
    with open("tweetSentBR_extracted/{}".format(goldpath)) as f:
        text = f.read()

    assert '\t' not in text 
    
    outtext = re.sub(r'(.+?) (.+)',r'\1\t\2', text)
    tabbed.update({goldpath:StringIO(outtext)})
    print(goldpath)
    
print('Unifying..')

header = ['id', 'premise']
abbr = {'neg': 'Negative', 'neu': 'Neutral', 'pos': 'Positive'} 
corpus = pd.DataFrame()

for path, f in tabbed.items():
    table = pd.read_csv(f, sep = '\t', names = header)

    posfix = path.split('.')[1]
    label = abbr[posfix]
    table['label'] = label

    corpus = table if corpus.empty else corpus.append(table)

Separating lines in tabs...
testTT.pos
testTT.neu
testTT.neg
Unifying..


In [0]:
mode = 'st-dnn' #'st-dnn', 'mt-dnn_assin+tweetsent'
pretrained = 'bert-pt'
filepath = f'../../report/{mode}/{pretrained}/tweetsent.json'

Gerar *scores*

In [20]:
from sklearn.metrics import f1_score
from importlib import reload  
from classify import report
import pandas as pd
import json

with open(filepath) as f:
    predictions = json.load(f)

number = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y_test = corpus['label'].map(number)


report(None, predictions, y_test)

Acc: 0.7458
F1. neg: 0.736 neu: 0.622 post: 0.825
Acc dist min: 0.2920
Acc dist max: 0.2920
Acc dist min equal: 0.0831
Acc dist max equal: 0.2090
